# pyflink在实际算法流程中的测试

我们模拟了一个实际数据表，in_loan_behavior。文件为：created_in_loan_behavior.csv。
重点解决了一个问题：

    使用flink加载csv时，需要提供csv文件中的列名和列类型。数据表可能有上百个列，如何获取这两组数据是一个难点。
    为此，我们使用了一个映射表，将数据库列类型映射为flinktable列类型。只要能获取到数据库schema就可以解决这个问题。
    并且，展示了如何使用flinktable加载csv文件的细节。

我们对pyflink table和pandas加载csv文件进行了比较测试。

我们发现，在加载csv文件上，使用pyflink大概比pandas直接读取要快10-100倍。
缺点是无法即时可视化。
使用建议是，flinktable进行多个处理步骤后，再使用to_pandas()查看。因为to_pandas()耗时较大。

In [1]:
from pyflink.table import *
from pyflink.table.types import *

environment_settings = EnvironmentSettings.new_instance().in_batch_mode().use_blink_planner().build()
t_env = BatchTableEnvironment.create(environment_settings=environment_settings)
t_env.get_config().get_configuration().set_string("parallelism.default", "1")

In [2]:
# 准备数据表schema
# 从数据表的字段名字和类型文件中获取二者的数组

def get_columns_and_types():
    processed_schema_file = "./dataset/IN_LOAN_BEHAVIOR_02.txt"
    column_names = []
    column_types = []
    with open(processed_schema_file) as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if i % 2 == 0:
                column_names.append(line.strip())
            else:
                column_types.append(line.strip())
    return column_names, column_types
            

column_names, column_types = get_columns_and_types()

In [3]:
column_names
len(column_names)

342

In [4]:
column_types
len(column_types)

342

In [5]:
column_types_set = set(column_types)
column_types_set

{'BIGINT', 'DATETIME', 'DECIMAL', 'DOUBLE', 'INT', 'VARCHAR'}

In [6]:
# 数据库列类型和pyflink类型映射表
types_map = {
    'INT':DataTypes.INT(),
    'FLOAT':DataTypes.FLOAT(),
    'DOUBLE':DataTypes.DOUBLE(),
    'BIGINT':DataTypes.BIGINT(),
    'DECIMAL':DataTypes.DECIMAL(38,18),
    'VARCHAR':DataTypes.STRING(),
    'DATE':DataTypes.DATE(), #日期
    'TIME':DataTypes.TIME(), #时间
    'DATETIME':DataTypes.TIMESTAMP(3), #日期+时间
}
types_map

{'INT': IntType(true),
 'FLOAT': FloatType(true),
 'DOUBLE': DoubleType(true),
 'BIGINT': BigIntType(true),
 'DECIMAL': DecimalType(38, 18, true),
 'VARCHAR': VarCharType(2147483647, true),
 'DATE': DateType(true),
 'TIME': TimeType(0, true),
 'DATETIME': TimestampType(3, true)}

In [7]:
#构建tablesource所需的两个数组：field_names, field_types
field_names = column_names
field_types = []
for column_type in column_types:
    field_types.append(types_map[column_type])

In [8]:
# field_types

In [9]:
# field_names

In [10]:
# 使用pyflink加载csv文件测试
source_csv_path = "./dataset/created_in_loan_behavior.csv"

import time
time1 = time.time()
table_source = CsvTableSource(source_path=source_csv_path, field_names=field_names, field_types=field_types, 
                              ignore_first_line=True)

table = t_env.from_table_source(table_source)
time2 = time.time()
print("load 3G file need: {}".format(time2 - time1))
# table.to_pandas().head(10)
# time3 = time.time()
# print("3G file trasfer to pandas: {}".format(time3 - time2)) # >= 10min

load 3G file need: 0.4451870918273926


In [11]:
# 使用pyfink table读取部分列测试
time1 = time.time()
table01 = table.select(table.ID, table.ACCT_NO, table.ID_NO, table.CIRCULATION_TAG, table.CURR_CIRCULATION_TAG)
# table = table.select(col("*"))
# table01.to_pandas()
time2 = time.time()
print("select and transfer need: {}".format(time2 - time1))

select and transfer need: 0.2020866870880127


In [12]:
# 使用pandas直接读取部分列测试
import pandas as pd
time1 = time.time()
usecols = ["ID", "ACCT_NO", "ID_NO", 'CIRCULATION_TAG',
 'CURR_CIRCULATION_TAG',]
# 注意csv文件中没有表头，需要设置header和names
df = pd.read_csv(source_csv_path, usecols=usecols, header=None, names=field_names)
time2 = time.time()
print("read csv need directly: {}".format(time2 - time1))
df

read csv need directly: 11.031697034835815


,ID,ACCT_NO,ID_NO,CIRCULATION_TAG,CURR_CIRCULATION_TAG
0,123456789,123,i love meituan,i love meituan,i love meituan
1,123456789,123,i love meituan,i love meituan,i love meituan
2,123456789,123,i love meituan,i love meituan,i love meituan
3,123456789,123,i love meituan,i love meituan,i love meituan
4,123456789,123,i love meituan,i love meituan,i love meituan
...,...,...,...,...,...
999995,123456789,123,i love meituan,i love meituan,i love meituan
999996,123456789,123,i love meituan,i love meituan,i love meituan
999997,123456789,123,i love meituan,i love meituan,i love meituan
999998,123456789,123,i love meituan,i love meituan,i love meituan
